<a href="https://colab.research.google.com/github/solvedbrunus/AB-Image-Classification/blob/main/mainCIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Preprocessing



In [ ]:
#this is coment 